In [50]:
from sklearn.datasets import fetch_california_housing 
from sklearn.model_selection import train_test_split 
from sklearn.preprocessing import StandardScaler

housing = fetch_california_housing()
X_train_full, X_test, y_train_full, y_test = train_test_split(housing.data,housing.target, random_state=42)
X_train, X_valid, y_train, y_valid = train_test_split(X_train_full, y_train_full, random_state=42)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_valid = scaler.transform(X_valid)
X_test = scaler.transform(X_test)

In [51]:
from scipy.stats import reciprocal
param_distribs = {
    "model__n_hidden": [0,1,2,3],
    "model__n_neurons": list(range(1,101)),
    "model__learning_rate": reciprocal(3e-4, 3e-2).rvs(1000).tolist(),
    "model__optimizer": ["Adam", "sgd", "nesterov"],
}

In [52]:
def build_model(n_hidden=1, n_neurons=30, optimizer="adam", learning_rate=0.001):
    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.InputLayer(shape=X_train.shape[1:]))
    
    for _ in range(n_hidden):
        model.add(tf.keras.layers.Dense(n_neurons, activation="relu"))
        
    model.add(tf.keras.layers.Dense(1))
    
    if optimizer == "sgd":
        opt = tf.keras.optimizers.SGD(learning_rate=learning_rate)
    elif optimizer == "nesterov":
        opt = tf.keras.optimizers.SGD(learning_rate=learning_rate, nesterov=True, momentum=0.9)
    elif optimizer == "momentum":
        opt = tf.keras.optimizers.SGD(learning_rate=learning_rate, momentum=0.9)
    else:
        opt = tf.keras.optimizers.Adam(learning_rate=learning_rate)
    
    model.compile(loss="mean_squared_error", optimizer=opt)
    return model


In [53]:
import tensorflow as tf
from scikeras.wrappers import KerasRegressor
es = tf.keras.callbacks.EarlyStopping(patience=10, min_delta=1.0, verbose=1)
keras_reg = KerasRegressor(build_model, callbacks=[es])

In [54]:
from sklearn.model_selection import RandomizedSearchCV

rnd_serach_cv = RandomizedSearchCV(keras_reg, param_distribs, n_iter=5, cv=3, verbose=2)

rnd_serach_cv.fit(X_train, y_train, epochs=100, validation_data=(X_valid, y_valid), verbose=0)

Fitting 3 folds for each of 5 candidates, totalling 15 fits
Epoch 14: early stopping
121/121 ━━━━━━━━━━━━━━━━━━━━ 0s 348us/step
[CV] END model__learning_rate=0.00032870009465034417, model__n_hidden=2, model__n_neurons=61, model__optimizer=sgd; total time=   1.7s
Epoch 26: early stopping
121/121 ━━━━━━━━━━━━━━━━━━━━ 0s 484us/step
[CV] END model__learning_rate=0.00032870009465034417, model__n_hidden=2, model__n_neurons=61, model__optimizer=sgd; total time=   3.0s
Epoch 14: early stopping
121/121 ━━━━━━━━━━━━━━━━━━━━ 0s 333us/step
[CV] END model__learning_rate=0.00032870009465034417, model__n_hidden=2, model__n_neurons=61, model__optimizer=sgd; total time=   1.7s
Epoch 11: early stopping
121/121 ━━━━━━━━━━━━━━━━━━━━ 0s 280us/step
[CV] END model__learning_rate=0.0010606173021654125, model__n_hidden=0, model__n_neurons=79, model__optimizer=nesterov; total time=   1.3s
Epoch 21: early stopping
121/121 ━━━━━━━━━━━━━━━━━━━━ 0s 281us/step
[CV] END model__learning_rate=0.0010606173021654125, mod

RandomizedSearchCV(cv=3,
                   estimator=KerasRegressor(callbacks=[<keras.src.callbacks.early_stopping.EarlyStopping object at 0x34eb753a0>], model=<function build_model at 0x34a382700>),
                   n_iter=5,
                   param_distributions={'model__learning_rate': [0.008164650734223027,
                                                                 0.019205370542684324,
                                                                 0.014611742381754998,
                                                                 0.0013057540305338037,
                                                                 0.0006308588359369814,
                                                                 0.00971277088602...
                                                                 0.010596533974296487,
                                                                 0.0008136255479886064,
                                                                 0.02165636377834367,
                                                                 0.002143665219415884,
                                                                 0.0007962934549136092,
                                                                 0.0009596009125799849,
                                                                 0.00030730763564978876,
                                                                 0.0005363122045620248, ...],
                                        'model__n_hidden': [0, 1, 2, 3],
                                        'model__n_neurons': [1, 2, 3, 4, 5, 6,
                                                             7, 8, 9, 10, 11,
                                                             12, 13, 14, 15, 16,
                                                             17, 18, 19, 20, 21,
                                                             22, 23, 24, 25, 26,
                                                             27, 28, 29, 30, ...],
                                        'model__optimizer': ['Adam', 'sgd',
                                                             'nesterov']},
                   verbose=2)

In [55]:
import pickle

best_param = rnd_serach_cv.best_params_

with open("rnd_search_params.pkl", "wb") as f:
    pickle.dump(best_param, f)
    
with open("rnd_search_scikeras.pkl", "wb") as f:
    pickle.dump(rnd_serach_cv, f)

In [56]:
import keras_tuner as kt
def build_model_kt(hp):
    n_hidden = hp.Int("n_hidden", min_value=0, max_value=3, default=2)
    n_neurons = hp.Int("n_neurons", min_value=16, max_value=256)
    learning_rate = hp.Float("learning_rate", min_value=1e-4, max_value=1e-2, sampling="log")
    
    optimizer = hp.Choice("optimizer", values=["sgd", "adam"])
    if optimizer == "sgd":
        optimizer = tf.keras.optimizers.SGD(learning_rate=learning_rate)
    else:
        optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)

    model = tf.keras.models.Sequential()
 
    model.add(tf.keras.layers.Flatten())
    
    for _ in range(n_hidden):
        model.add(tf.keras.layers.Dense(n_neurons, activation="relu"))

    model.add(tf.keras.layers.Dense(1))
    
    model.compile(loss="mse", optimizer=optimizer, metrics=["mse"])
    return model 

In [57]:
random_search_tuner = kt.RandomSearch(
    build_model_kt, 
    objective="val_mse",  
    max_trials=10, 
    overwrite=True,
    directory="my_california_housing", 
    project_name="my_rnd_search", 
    seed=42
)
    

In [58]:
import os
root_logdir = os.path.join(random_search_tuner.project_dir, 'tensorboard')
tb = tf.keras.callbacks.TensorBoard(root_logdir)

In [59]:
random_search_tuner.search(X_train, y_train, epochs=100, callbacks=[es, tb],
validation_data=(X_valid, y_valid))
best_hyperparams = random_search_tuner.get_best_hyperparameters()[0]

Trial 10 Complete [00h 00m 04s]
val_mse: 0.29301711916923523

Best val_mse So Far: 0.28973594307899475
Total elapsed time: 00h 00m 43s


In [60]:
final_dict = {
    'n_hidden': best_hyperparams.get('n_hidden'),
    'n_neurons': best_hyperparams.get('n_neurons'),
    'learning_rate': best_hyperparams.get('learning_rate'),
    'optimizer': best_hyperparams.get('optimizer')
}

In [61]:
with open("kt_search_params.pkl", "wb") as f:
    pickle.dump(final_dict, f)

In [62]:
best_model = random_search_tuner.hypermodel.build(best_hyperparams)

best_model.compile(loss="mse", 
                   optimizer=best_model.optimizer, 
                   metrics=["mse"])

best_model.fit(X_train, y_train, 
               epochs=100, 
               validation_data=(X_valid, y_valid), 
               callbacks=[es, tb])

Epoch 1/100
363/363 ━━━━━━━━━━━━━━━━━━━━ 1s 880us/step - loss: 1.3665 - mse: 1.3665 - val_loss: 0.4054 - val_mse: 0.4054
Epoch 2/100
363/363 ━━━━━━━━━━━━━━━━━━━━ 0s 764us/step - loss: 0.3804 - mse: 0.3804 - val_loss: 0.3539 - val_mse: 0.3539
Epoch 3/100
363/363 ━━━━━━━━━━━━━━━━━━━━ 0s 755us/step - loss: 0.3405 - mse: 0.3405 - val_loss: 2.9284 - val_mse: 2.9284
Epoch 4/100
363/363 ━━━━━━━━━━━━━━━━━━━━ 0s 755us/step - loss: 0.3401 - mse: 0.3401 - val_loss: 3.6727 - val_mse: 3.6727
Epoch 5/100
363/363 ━━━━━━━━━━━━━━━━━━━━ 0s 868us/step - loss: 0.3452 - mse: 0.3452 - val_loss: 0.4874 - val_mse: 0.4874
Epoch 6/100
363/363 ━━━━━━━━━━━━━━━━━━━━ 0s 830us/step - loss: 0.3278 - mse: 0.3278 - val_loss: 2.9872 - val_mse: 2.9872
Epoch 7/100
363/363 ━━━━━━━━━━━━━━━━━━━━ 0s 799us/step - loss: 0.3066 - mse: 0.3066 - val_loss: 1.1049 - val_mse: 1.1049
Epoch 8/100
363/363 ━━━━━━━━━━━━━━━━━━━━ 0s 783us/step - loss: 0.3085 - mse: 0.3085 - val_loss: 0.8587 - val_mse: 0.8587
Epoch 9/100
363/363 ━━━━━━━━━━━━

In [63]:
best_model.save("kt_best_model.keras")